In [5]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

#### Retorna a categoria do local

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

#### Informações para acessar a API do foursquare

In [7]:
CLIENT_ID = '55LWIFZLZNZPSTAZZQKJXBD11BSBCUZTZPIXWVV4RFZJP1HS' # your Foursquare ID
CLIENT_SECRET = 'RPCOWGHQY5YNQUCHQUTC5VLKEYFGZN15FSIYTK5GNXNZ301E' # your Foursquare Secret
VERSION = '20180604'

#### Informações da cidade delimitada

In [8]:
latitude_nyork = 40.730610
longitude_nyork = -73.935242
radius = 5000
LIMIT = 1000
QUERY = 'beer'

#### Coletando as informações dos locais

In [9]:
url = 'https://api.foursquare.com/v2/venues/search?intent=checkin&query={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    QUERY, CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()

In [10]:
venues = results['response']['venues'] 
nearby_venues = json_normalize(venues)
filtered_columns = ['id', 'name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.tail(10)

,id,name,categories,lat,lng
40,4c015ce80bce20a1a098f6c6,21st St Beer And Soda,Liquor Store,40.768274,-73.931118
41,57339ff5498ebbf28b9eb4a2,Beer Garden Dazzler Brooklyn Hotel,Beer Garden,40.696674,-73.984278
42,50ba7adbe4b036209ec2a3bd,Big Gay Ice Cream Shop,Ice Cream Shop,40.733193,-74.002988
43,5b12decdf00a70002c018513,Wyckoff Beer Garden,Beer Garden,40.706304,-73.922266
44,4cccb47d511b236a6d34f3c9,69st beer & soda,None,40.716611,-73.957280
45,4f43f88e19836ed001917b1f,Sixty-Ninth St Beer Distributors,Beer Store,40.727403,-73.895335
46,513d2416e4b02357ebfb439e,Avenue C Beer Company,Beer Garden,40.722835,-73.979798
47,559472f0498eb09835d77213,Broke Hustler's Brewering Beer Garden,Beer Garden,40.765965,-73.911443
48,57658a11498e91e51599a8dc,C & P Beer Distributors,Warehouse Store,40.704480,-73.928002
49,571abec5498e9a18b35e43d4,Bowery Beer Garden,Beer Garden,40.717192,-73.995312


In [93]:
users_total = pd.DataFrame()
for venue in nearby_venues.iterrows():
    url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue[1]['id'], CLIENT_ID, CLIENT_SECRET, VERSION, 2000)
    results = requests.get(url).json()
    if (results['response']['likes']['count'] > 0) & ('items' in results['response']['likes']):
        results_2 = results['response']['likes']['items']
        users = json_normalize(results_2)[['id', 'firstName', 'gender']]
        users['avenue_id'] = venue[1]['id']
        users['avenue_name'] = venue[1]['name']
        users['avenue_categories'] = venue[1]['categories']
        users['avenue_lat'] = venue[1]['lat']
        users['avenue_lng'] = venue[1]['lng']
        if users_total.empty:
            users_total = users
        else:
            users_total = pd.concat([users_total, users], ignore_index=True)

In [94]:
users_total.tail()

,id,firstName,gender,avenue_id,avenue_name,avenue_categories,avenue_lat,avenue_lng
9257,39991457,Jannis,female,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9258,6909884,Carol 🎄⛄,female,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9259,3088472,Chase,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9260,14184080,John,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9261,2631756,Tokuyuki,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476


In [95]:
users_total.to_csv('curtidas_cervejarias.csv', index=False)